# Extract External catalogs

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/29
- update : 2024/03/04


Access to the external catalog

``quantum graphs ``: https://tigress-web.princeton.edu/~lkelvin/pipelines/w_2024_07/drp_pipe/LATISS/DRP/pipeline_drp_pipe_LATISS_DRP_step1.pdf

In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

## Configuration

In [ ]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry
skymap_auxtel ='latiss_v1'

In [ ]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [ ]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Tracts and Patches

In [ ]:
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=my_collection)

In [ ]:
ls ../data/202402

In [ ]:
# Load data (deserialize)
import pickle
#with open('Latiss_tractspatchs.pickle', 'rb') as handle:
with open('../data/202402/Latiss_tractspatchs_radec_htm7_skypixels_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

In [ ]:
all_skypixels = []
for key, value in tracts_patchs_dict.items():
    all_skypixels.append(value)
all_skypixels = np.concatenate(all_skypixels)
all_sypixels= np.unique(all_skypixels)
all_skypixels = np.sort(all_skypixels)
all_skypixels  
NPIX = len(all_skypixels)

# Reference Sources

## Dump

In [ ]:
#data_product = "gaia_dr2_20200414" 
#data_product = "gaia_dr3_20230707"
data_product = 'atlas_refcat2_20220201'

datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
    if i >0:
        break

In [ ]:
registry.getDatasetType(data_product)

In [ ]:
print(butler.registry.dimensions['htm7'].RecordClass.fields)

In [ ]:
for tractID in tracts_patchs_dict.keys():
    List_of_Skypixels = tracts_patchs_dict[tractID]
    print(tractID,  List_of_Skypixels)
    for htm7_id in List_of_Skypixels:
        where_clause = f"instrument='LATISS' AND detector=0 AND htm7={htm7_id}"
        #where_clause = f"instrument='LATISS' AND detector=0"
        dataId = {'htm7': htm7_id}
        print("ask for where and dataId : ",where_clause,dataId)
        datasetRefs = butler.registry.queryDatasets(datasetType=data_product,  dataId=dataId,collections=my_collection,where=where_clause)
        #datasetRefs = registry.queryDatasets(data_product, dataId=dataId)
        print("fullId.............................:",ref.dataId.full)

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")

In [ ]:
all_df = []

count = 0
for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
  
    skypix = ref.dataId['htm7']
    if skypix not in all_skypixels:
        continue

    if count> NPIX:
        break
        
    t = butler.get(ref).asAstropy()
    df = t.to_pandas()
    #df.rename(columns={"coord_ra": "ra", "coord_dec": "dec"}, errors="raise",inplace=True)
    #df[["tractID","patchID"]] = df.apply( FindTractAndPatch, axis=1,result_type ='expand')
    #if(len(df)>0):
    #print(type(df),len(df))
    df['skypix'] = skypix
    all_df.append(df)
    count += 1

 
    

In [ ]:
df.columns

In [ ]:
df = pd.concat(all_df)

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())